In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])

In [2]:
ind = "network_weather_2-2015.12.12"

my_query = \
{
  "size": 0,
  "aggs": {      
        "type_counts": {
            "terms": { "field": "_type" },
            "aggs": {      
                "src_counts": {
                     "terms": { "field": "src" , "size": 10000},
                     "aggs": {      
                        "dest_counts": {
                        "terms": { "field": "dest" , "size": 10000}
                        }
                     }
                  }
              }
        }
  }
}

In [3]:
response = es.search(index=ind, body=my_query, request_timeout=600)

In [ ]:
#print(response['aggregations']['type_counts']['buckets'])


In [ ]:

types = response['aggregations']['type_counts']['buckets']

for t in types:
    print(t['key'])
   # if t['key'] != 'packet_loss_rate': continue
    for s in t['src_counts']['buckets']:
        for d in s['dest_counts']['buckets']:
            src=s['key']
            dest=d['key']
            print ('s: ',src,'d: ', dest, '\t docs:', d['doc_count'])
            myscroll={
            "size":100000,
#             "fields": ["timestamp", "delay_mean","throughput","packet_loss","src","dest"],
            "fields": ["timestamp","_source"],
            "query":{
                "filtered": {
                    "query": {
                         "match_all": {} 
                        },
                        "filter" : { 
                            "bool" : {
                                "must" : [
                            {"term": {"_type": {"value": t['key'] } } },
                            {"term": {"src":   {"value": src } } },
                            {"term": {"dest":  {"value": dest } } }
                                ]
                            }

                        }
                }
            }
            ,"sort": [{ "timestamp": { "order": "asc" }}]
            }
            
            toKeep=[]
            hashes=[]
            counter=0
            scroll = scan(es, query=myscroll, index=ind, scroll='120s')
            for res in scroll:
#                 if counter==0: print(res)
                if res['fields']['timestamp'][0] not in hashes:
                    toKeep.append(res['_source'])
#                     print(res['fields'])
#                 oldres=res['fields']['packet_loss'][0]
                oldtime=res['fields']['timestamp'][0]
                counter+=1
                if counter>200:
                   break
#                 if counter%100==0: 
                
            print ('\tto keep:',len(toKeep))


latency
s:  85.122.31.74 d:  146.83.90.7 	 docs: 4824
{'_score': 0.0, '_index': 'network_weather_2-2015.12.12', 'fields': {'timestamp': ['2015-12-12T07:05:00']}, '_type': 'latency', '_id': 'AVGVBvWk2PN9pzsFyNxO', '_source': {'destSite': 'EELA-UTFSM', 'delay_median': 139.2, 'timestamp': '2015-12-12T07:05:00', 'MA': '85.122.31.74', 'srcSite': 'RO-16-UAIC', 'destVO': 'ATLAS', 'srcVO': 'ATLAS', 'src': '85.122.31.74', 'srcProduction': True, 'dest': '146.83.90.7', 'destProduction': True, 'delay_mean': 140.2595642701525, 'delay_sd': 18.981688545227065}}
records: 201 	to keep: 201
s:  85.122.31.74 d:  128.40.4.25 	 docs: 4356
{'_score': 0.0, '_index': 'network_weather_2-2015.12.12', 'fields': {'timestamp': ['2015-12-12T05:45:00']}, '_type': 'latency', '_id': 'AVGUu_jO2PN9pzsFTQDI', '_source': {'destSite': 'UKI-LT2-UCL-HEP', 'delay_median': 21.2, 'timestamp': '2015-12-12T05:45:00', 'MA': '85.122.31.74', 'srcSite': 'RO-16-UAIC', 'destVO': 'ATLAS', 'srcVO': 'ATLAS', 'src': '85.122.31.74', 'srcPro

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='atlas-kibana.mwt2.org', port=9200): Read timed out. (read timeout=10))